## Number of hEs points per hour as a function of average/peak riometer absorption in that hour

#### 1) Get average riometer absorption per hour for the days:
##### 1.  2012-01-23
##### 2.  2012-02-07
##### 3.  2012-03-01
##### 4.  2012-03-07
##### 5.  2012-03-08
##### 6.  2012-03-13
##### 7.  2012-03-15
##### 8.  2012-04-03
##### 9.  2012-04-25
##### 10. 2012-07-06
##### 11. 2012-11-01
##### 12. 2012-11-07

#### 2) Get hEs points per hour for those days
#### 3) Plot

In [1]:
# 1) average rio absorption

In [2]:
import datetime
import urllib.request

# define riometer readfile function
def rio_readfile(url):

    # Define lists
    #date = [] #do i even need the date cause it's the same year?
    time = []
    absorption = []
    raw_sig = []

    # Define filename
    #filename = "RD 2012-03-03.txt"

    # open file to read
    response = urllib.request.urlopen(url)
    html_response = response.read()
    encoding = response.headers.get_content_charset("utf-8")
    fp = html_response.decode(encoding)

    # define new list sanitized_data
    # entry = [] list defined later, append datetime, absp, raw sig to it
    # later append entry to sanitized_data so it will be lists within a list
    sanitized_data = []
    
    for line in fp.splitlines():
        #print(line)
        #print(str(line))
    
        # skip comments
        if line[0] == "#":
            continue
        else:
            
            # strip line
            line_strip = line.strip()
            # split lines into lists
            line_split = line.split()
            #print(line_split[1])

            # Define datetime format for date and time
            
            format = "%d%m%Y%H:%M:%S"
            
            # split column 0 to month, date, year and make one row
            month = str(line_split[0].split("/")[1])
            day = str(line_split[0].split("/")[0])
            year = "20" + str(line_split[0].split("/")[2])
                      
            # change this to dd mm yy!!!!!!!!!!!
            full_date = day + month + year + str(line_split[1])
            
            # Try-except to see if can convert to datetime
            try:
                res = bool(datetime.datetime.strptime(full_date, format))
                this_time = datetime.datetime.strptime(full_date, format)

                # Get rid of negative absorption values
                if float(line_split[2]) < 0:
                    continue
                # append time to array
                time.append(this_time)
                # append absorption to array
                absorption.append(float(line_split[2]))
                # append raw signals to array
      
                raw_sig.append(line_split[3])

                # define new list 
                entry = []
                entry.append(this_time)
                entry.append(line_split[2])
                entry.append(line_split[3])
                
                sanitized_data.append(entry)
                    
            except ValueError:
                res = False

    #return sanitized_data
    return sanitized_data, time, absorption, raw_sig
    #return absorption
    #return raw_sig

In [3]:
## Import stuff
import numpy as np

def ionosonde_plotter(f):
    # Define timestamp array
    timestamp_full = []

    # Minimum virtual height of E trace
    hE = []
    timestamp_hE = []
    # Minimum virtual height of Es trace
    hEs = []
    timestamp_hEs = []

    # Define CS array
    CS = []

    # E layer critical frequency
    foE =[]
    timestamp_foE = []
    # Es layer critical frequency
    foEs = []
    timestamp_foEs = []


    # Define filename
    filename = f + ".txt"

    # define indices for each parameter of file
    time_idx = 0
    CS_idx = 1
    foEs_idx = 2
    foE_idx = 4
    hE_idx = 6
    hEs_idx = 8

    # Define format for datetime
    format = "%Y-%m-%dT%H:%M:%S.%fZ"

    # Open file - read
    fp = open(filename, 'r')
    # Read every line in file
    for line in fp:
    # Skip comments
        if line[0] == "#":
            continue
    # Strip lines
        else:
            line_strip = line.strip()
    # Split lines into lists
            line_split = line.split()

            foE_str = line_split[foE_idx]
            foEs_str = line_split[foEs_idx]
            hE_str = line_split[hE_idx]
            hEs_str = line_split[hEs_idx]

            timestamp_full.append(datetime.datetime.strptime(line_split[time_idx], format))

            # Check if can convert foE to float
            try:
                foE_float = float(foE_str)
                # If successful, append to the array
                foE.append(foE_float)
                # Extract timestamp and append to the array
                timestamp_str = line_split[time_idx]
                timestamp_foE.append(datetime.datetime.strptime(timestamp_str, format))
            except ValueError:
                # If the conversion fails, ignore the line
                pass

            # Check if can convert foEs to float
            try:
                foEs_float = float(foEs_str)
                # If successful, append to the array
                foEs.append(foEs_float)
                # Extract timestamp and append to the array
                timestamp_str = line_split[time_idx]
                timestamp_foEs.append(datetime.datetime.strptime(timestamp_str, format))
            except ValueError:
                # If the conversion fails, ignore the line
                pass

            # Check if hE can convert to float
            try:
                hE_float = float(hE_str)
                # If successful, append to the array
                hE.append(hE_float)
                # Extract timestamp and append to the array
                timestamp_strhE = line_split[time_idx]
                timestamp_hE.append(datetime.datetime.strptime(timestamp_strhE, format))
            except ValueError:
                # If the conversion fails, ignore the line
                pass

            # Check if hEs can convert to float
            try:
                hEs_float = float(hEs_str)
                # If successful, append to the array
                hEs.append(hEs_float)
                # Extract timestamp and append to the array
                timestamp_strhEs = line_split[time_idx]
                timestamp_hEs.append(datetime.datetime.strptime(timestamp_strhEs, format))
            except ValueError:
                # If the conversion fails, ignore the line
                pass

    return timestamp_foEs, foEs, timestamp_hEs, hEs


In [4]:
# call rio readfile on 2012/03/08 riometer file
url = "https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/08/norstar_k2_rio-daws_20120308_v01.txt"

sanitized_data, time, absorption, raw_sig = rio_readfile(url)

In [5]:
# call ionosonde readfile on 2012-03-08
f = "2012-03-08"

timestamp_foEs, foEs, timestamp_hEs, hEs = ionosonde_plotter(f)

#### Test 

In [6]:
import statistics as stats

hour = 2
abs_values = []

for idx, list in enumerate(sanitized_data):
    time_needed = list[0]
    t_hour = int(time_needed.hour)
    if t_hour == hour:
        abs_needed = float(list[1])
        abs_values.append(abs_needed)
        avg = stats.mean(abs_values)

print(avg)

5.287029166666667


### Average absorption values for each hour of one UT day

In [7]:
hours = []
for i in range(0,24):
    hours.append(i)

# list to store average absorption values for each hour of the day (length = 24)
avg_abs = []

# go through all 24 hours (from 0-23)
for idx, list in enumerate(sanitized_data):
    # go through each list in san_data
    for hour in hours:
        # get timestamp from list
        time_needed = list[0]
        # convert it to int hour
        t_hour = int(time_needed.hour)
        # define list to store indices of correct lists for that hour
        index_hour_list = []
        # if the hours match
        if t_hour == hour:
            index_list.append(idx)

        for i in index_list:
            needed_list = sanitized_data[i]
            # try except on absorption vals to float
            try:
                # get the absorption values and convert to float
                abs_needed = float(needed_list[1])
                # append the absorption needed to abs_values list
                abs_values.append(abs_needed)
            except ValueError:
                pass

        # find avg of that list
        avg = stats.mean(abs_values)
    # append that avg to avg_abs list
    avg_abs.append(avg)

print(avg_abs)

KeyboardInterrupt: 